## Creación de un csv con las estadísticas de los jugadores de la nba durante la temporada 24/25
Primero instalamos nba_api, para poder acceder a los datos y crear el csv

In [1]:
# Este paso no tenemos por qué hacerlo aquí, podríamos hacerlo en la terminal de linux.
!pip install nba_api

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


### Ahora toca crear el csv como tal
Para ello importamos pandas y la nba api y seguimos los pasos del fragmento de código siguiente:

In [2]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats

## Cargamos las estadísticas a una variable stats
A partir de esta variable vamos a hacer un dataset con las estadísticas de la temporada que indiquemos (no tendría porque ser de la 24/25)

In [3]:
# Descargamos estadísticas generales (incluye avanzadas y básicas)
# Vamos a pillar los datos de la temporada 2024-25, ya que es la más reciente y que ha acabado, en caso de poner la 25/26 faltarían datos.
# He estado probando y la fecha más antigua que me funciona es la 1996-97
stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2024-25', measure_type_detailed_defense='Base')
df = stats.get_data_frames()[0]

### Guía de datos:
- PLAYER_NAME: Nombre del jugador
- TEAM_ABBREVIATION: Abreviatura del equipo
- AGE: Edad del jugador
- GP: Partidos jugados
- W: Victorias
- L: Derrotas
- MIN: Minutos jugados
- PTS: Puntos por partido
- REB: Rebotes por partido
- AST: Asistencias por partido
- STL: Robos por partido
- BLK: Tapones por partido
- FG_PCT: Porcentaje de tiros de campo
- FG3_PCT: Porcentaje de triples
- FT_PCT: Porcentaje de tiros libres
- PLUS_MINUS: Plus/Minus del jugador
- NBA_FANTASY_PTS: Puntos del fantasy
- DD2: Dobles-dobles
- TD3: Triples-dobles

Puse esto por si queremos hacer otra operación con el dataset o la persona que vea esto tiene alguna duda sobre que estadísticas estamos recogiendo.

## Seleccionamos los datos deseados
Vamos a seleccionar los datos que se mencionan en la guía de datos, pero podríamos seleccionar más datos, ya que la api tiene más alcance que esto
- Aquí está el github de la api: [NBA_API](https://github.com/swar/nba_api)

In [4]:
# Seleccionamos los datos (algunos básicos y algunos avanzados)
columnas_deseadas = [
    'PLAYER_NAME', 'TEAM_ABBREVIATION', 'AGE', 'GP', 'W', 'L', 'MIN',
    'PTS', 'REB', 'AST', 'STL', 'BLK',
    'FG_PCT', 'FG3_PCT', 'FT_PCT',
    'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3' 
]

# Y guardamos el dataset
dataset_final = df[columnas_deseadas]

## Y solo queda importarlo como csv!

In [5]:
# Guardamos a CSV
dataset_final.to_csv('source_clean_dataset.csv', index=False)
print("¡Listo! Archivo 'source_clean_dataset.csv' creado con éxito.")

¡Listo! Archivo 'source_clean_dataset.csv' creado con éxito.


# Y ahora toca ensuciar los datos!
Para ello usaremos el data set creado anteriormente (obviamente).

### Importamos lo necesario para poder ensuciar los datos del csv

In [6]:
import numpy as np
import random

### Ahora cargamos el csv limpio para ensuciarlo

In [7]:
dfSucio = pd.read_csv('source_clean_dataset.csv')

### Eliminar datos
En esta parte eliminaremos datos aleatoriamente de las columnas indicadas, en un principio borraremos un 5% de datos

In [8]:
columnasBorrarDatos = ['AGE', 'PTS', 'FG_PCT']
for cols in columnasBorrarDatos:
    mask = np.random.random(len(df)) < 0.05  # 5%
    dfSucio.loc[mask, cols] = np.nan

### Filas duplicadas
Pillaremos 20 filas aleatorias y las añadimos al final

In [9]:
duplicados = dfSucio.sample(n=20, random_state=42)
dfSucio = pd.concat([dfSucio, duplicados], ignore_index=True)

### Creación de outliers
Insertaremos valores outliers en minutos jugados (MIN) y en partidos jugados (GP)

In [10]:
outliersIndicesMin = np.random.choice(dfSucio.index, size=5, replace=False)
dfSucio.loc[outliersIndicesMin, 'MIN'] = 50000

outliersIndicesGP = np.random.choice(dfSucio.index, size=5, replace=False)
dfSucio.loc[outliersIndicesGP, 'GP'] = -10

### Formato inconsistente
Aquí vamos a cambiar el separador decimal '.' por ',' en FT_PCT, así lo convertimos a texto

In [11]:
# Con este código nos aseguramos de que FT_PCT sea tipo object
dfSucio['FT_PCT'] =dfSucio['FT_PCT'].astype(object)

comaIndices = np.random.choice(dfSucio.index, size=30, replace=False)
for idx in comaIndices:
    val=dfSucio.loc[idx, 'FT_PCT']
    if pd.notnull(val) and isinstance(val, (int, float)):
        dfSucio.loc[idx, 'FT_PCT'] = str(val).replace('.', ',')

### Errores tipográficos
Introducimos caracteres extraños en las variables categóricas

In [12]:
def fastidiarStrings(s):
    if pd.isna(s): return s
    s_list = list(s)
    if len(s_list) > 0:
        pos = random.randint(0, len(s_list) - 1)
        # Esto remplaza una letra al azar por un símbolo
        s_list[pos] = random.choice(['X', '?', '!'])
    return ''.join(s_list)

fallosIndices = np.random.choice(dfSucio.index, size=20, replace=False)
for idx in fallosIndices:
    dfSucio.loc[idx, 'TEAM_ABBREVIATION'] = fastidiarStrings(dfSucio.loc[idx, 'TEAM_ABBREVIATION'])

### Añadir un equipo extra
Vamos a añadir un equipo invalido

In [13]:
equipoExtra = np.random.choice(dfSucio.index, size=10, replace=False)
dfSucio.loc[equipoExtra, 'TEAM_ABBREVIATION'] = 'EquipoInvalido'

### Datos incorrectos y puntuación demás
Añadimos el texto "pts" a la columna PTS, haciendo que se convierta en un string

In [14]:
dfSucio['PTS'] = dfSucio['PTS'].astype(object)
PuntosIndicesMal = np.random.choice(dfSucio.index, size=40, replace=False)
for idx in PuntosIndicesMal:
    val = dfSucio.loc[idx, 'PTS']
    if pd.notnull(val):
        dfSucio.loc[idx, 'PTS'] = f"{val} puntos"

### Headers erroneos
Renombramos algunas columnas con algun espacio o símbolo

In [15]:
dfSucio.rename(columns={
    'NBA_FANTASY_PTS': 'NBA FANTASY POINTS',
    'PLUS_MINUS': '+/-',
    'PLAYER_NAME': 'PLAYER NAME',
}, inplace=True)

### Guardar el dataset sucio
Usamos 'cp1252' o 'latin1' para simular problemas de encoding (UTF-8 es el estándar)

In [ ]:
dfSucio.to_csv('dirty_dataset.csv', index=False, encoding='cp1252', errors='replace')
print("¡Listo! Archivo 'dirty_dataset.csv' creado con éxito.")

¡Listo! Archivo 'dirty_datase.csv' creado con éxito.
